In [164]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

# fetch dataset 
chess_king_rook_vs_king = fetch_ucirepo(id=23) 
  
# data (as pandas dataframes) 
X = chess_king_rook_vs_king.data.features 
y = chess_king_rook_vs_king.data.targets 
  


### Pozbycie się liter

In [165]:
df = pd.concat([X,y],axis=1)
słowa2num ={'draw':-1, 'zero':0, 'one':1, 'two':2, 'three':3, 'four':4, 'five':5, 'six':6,
       'seven':7, 'eight':8, 'nine':9, 'ten':10, 'eleven':11, 'twelve':12, 'thirteen':13,
       'fourteen':14, 'fifteen':15, 'sixteen':16}
litery2num = {"a": 1,"b":2,"c":3,"d":4,"e":5,"f":6,"g":7,"h":8}
df.loc[:,"white-depth-of-win"] = df.loc[:,"white-depth-of-win"].map(słowa2num)
df.loc[:,"black-king-file"] = df.loc[:,"black-king-file"].map(litery2num)
df.loc[:,"white-king-file"] = df.loc[:,"white-king-file"].map(litery2num)
df.loc[:,"white-rook-file"] = df.loc[:,"white-rook-file"].map(litery2num)
df=df.astype("float64")

# Zadanie 1 :

### Podział danych i tworzenie punktów do ładnej szachownicy

In [166]:
df1 = df.loc[df["white-depth-of-win"].isin([0,-1]),:]
df11 = df.loc[df["white-depth-of-win"]==0,:]
df12 = df.loc[df["white-depth-of-win"]==-1,:]

In [167]:
szachx = np.arange(1,9)
szachx= np.repeat(szachx,4)
szachy = np.array([1,3,5,7,2,4,6,8])
szachy = np.tile(szachy,4)


### Wykres 1

In [168]:
fig = go.Figure()

fig = make_subplots(
    rows=1, cols=2,
    horizontal_spacing=0.18
)
# Szachownica 1
fig.add_trace(go.Histogram2d(
    x=szachx,
    y=szachy,
    xbins=dict(start=0.5, end=8.5, size=1),
    ybins=dict(start=0.5, end=8.5, size=1),
    colorscale=[
        [0, '#F0D9B5'],
        [1, "#B58863"]
    ],
    showscale=False    
),row=1, col=1)


fig.add_trace(go.Histogram2d(
    x=df11["black-king-file"],
    y=df11["black-king-rank"],
    xbins=dict(start=0.5, end=8.5, size=1),
    ybins=dict(start=0.5, end=8.5, size=1),
    colorscale=[
        [0, 'rgba(0,0,0,0)'],
        [0.0001, "#D88181"],
        [1, "#ff0000"]
    ],
    showscale=True,
    colorbar=dict(
    x=0.44
)
),row=1, col=1)

# Szachownica 2
fig.add_trace(go.Histogram2d(
    x=szachx,
    y=szachy,
    xbins=dict(start=0.5, end=8.5, size=1),
    ybins=dict(start=0.5, end=8.5, size=1),
    colorscale=[
        [0, '#F0D9B5'],
        [1, "#B58863"]
    ],
    showscale=False    
),row=1, col=2)

fig.add_trace(go.Histogram2d(
    x=df12["black-king-file"],
    y=df12["black-king-rank"],
    xbins=dict(start=0.5, end=8.5, size=1),
    ybins=dict(start=0.5, end=8.5, size=1),
    opacity=0.9,
    colorscale=[
        [0, 'rgba(0,0,0,0)'],
        [0.0001, "#FFFFFF"],
        [1, "#2f00ff"]
    ],
    showscale=True,
    colorbar=dict(
    x=1.03
)
),row=1, col=2)


x_vals = [1, 2,3,4,5,6,7,8]
x_text = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
y_vals = [1, 2, 3, 4, 5, 6, 7, 8]
y_text = ['1', '2', '3', '4', '5', '6', '7', '8']


fig.update_xaxes(
    tickmode='array', tickvals=x_vals, ticktext=x_text,
    showgrid=False, zeroline=False, constrain='domain',
    ticklen=2, ticks="outside", tickcolor='rgba(0,0,0,0)',
    row=1, col=1
)
fig.update_yaxes(
    tickmode='array', tickvals=y_vals, ticktext=y_text,
    showgrid=False, zeroline=False, 
    ticklen=2, ticks="outside", tickcolor='rgba(0,0,0,0)',
    scaleanchor="x", scaleratio=1,
    row=1, col=1 
)

fig.update_xaxes(
    tickmode='array', tickvals=x_vals, ticktext=x_text,
    showgrid=False, zeroline=False, constrain='domain',
    ticklen=2, ticks="outside", tickcolor='rgba(0,0,0,0)',
    row=1, col=2
)
fig.update_yaxes(
    tickmode='array', tickvals=y_vals, ticktext=y_text,
    showgrid=False, zeroline=False,
    ticklen=2, ticks="outside", tickcolor='rgba(0,0,0,0)',
    scaleanchor="x2", scaleratio=1,
    row=1, col=2
)

fig.update_layout(
    title="Gdzie zazwyczaj znajduje się czarny król, gdy jest matowany oraz gdy dochodzi do remisu?",
    width=1200, height=600,
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False
)
fig.show()



### Czarny król jest matowany, gdy znajduje się w lewym dolnym rogu szachownicy z perspekty grającego po stronie białej (kolejność numercji pól). Do remisu dochodzi gdy jest po prawej części szachownicy, jednakże częściej się to dzieje 1-2 kratki od krawędzi.



# Zadanie 2 :

### Utworzenie kolumn z metrykami

In [169]:
df2 = df.loc[df.loc[:,"white-depth-of-win"]!=-1,:].copy()

In [170]:
x1,y1 = df2.loc[:,"white-king-file"].to_numpy(), df2.loc[:,"white-king-rank"].to_numpy()
x2,y2 = df2.loc[:,"black-king-file"].to_numpy(), df2.loc[:,"black-king-rank"].to_numpy()
df2.loc[:,"Euklides"] = np.sqrt((x1-x2)**2 +(y1-y2)**2)
df2.loc[:,"Manhattan"] = abs(x1-x2) + abs (y1-y2)
df2.loc[:,"Czebyszew"] = np.maximum(abs(x1-x2),abs(y1-y2))
df2.loc[:,"Do krawędzi"] = np.minimum(x2-1,np.minimum(8-x2,np.minimum(y2-1,8-y2)))
df3 = df2.copy()

In [171]:
df2 = pd.DataFrame(df2.groupby("white-depth-of-win")[["Euklides","Manhattan","Czebyszew","Do krawędzi"]].median()).reset_index()

In [172]:
df2

,white-depth-of-win,Euklides,Manhattan,Czebyszew,Do krawędzi
0,0.0,2.000000,2.0,2.0,0.0
1,1.0,2.236068,3.0,2.0,0.0
2,2.0,2.000000,2.0,2.0,0.0
3,3.0,2.236068,3.0,2.0,0.0
4,4.0,3.000000,3.0,3.0,0.0
5,5.0,3.000000,3.0,3.0,0.0
6,6.0,3.162278,4.0,3.0,0.0
7,7.0,3.605551,5.0,3.0,0.0
8,8.0,3.605551,5.0,3.0,0.0
9,9.0,4.242641,5.0,4.0,0.0


# Wykres 2

In [173]:
df2 = df2.melt(
    id_vars=['white-depth-of-win'],
    value_vars=['Euklides', 'Manhattan','Czebyszew','Do krawędzi'],
    var_name='Metryka',
    value_name='Odległość'
)

In [174]:


fig = px.scatter(
    df2, 
    x="white-depth-of-win", 
    y="Odległość", 
    color="Metryka",
    trendline="ols",
    title="Wykres z liniami trendu",
    width=1200
)
fig.update_traces(marker=dict(size=10))

fig.show()

In [175]:
dopasowania = px.get_trendline_results(fig)
dopasowania["R2"] = dopasowania["px_fit_results"].apply(lambda x: x.rsquared)
print(dopasowania[["Metryka", "R2"]].sort_values(by="R2", ascending=False))

       Metryka        R2
1    Manhattan  0.921601
0     Euklides  0.916292
2    Czebyszew  0.813225
3  Do krawędzi  0.611413


### Dla median z danych względem ilości ruchów do końca partii, metryki rosną wraz z odległością od mata, jednakże najlepszy współczynnik dopasowania do prostej ma z niewielką przewagą metryka Manhattan.

# Zadanie 3 :

In [176]:
df3

,white-king-file,white-king-rank,white-rook-file,white-rook-rank,black-king-file,black-king-rank,white-depth-of-win,Euklides,Manhattan,Czebyszew,Do krawędzi
2796,3.0,1.0,1.0,3.0,1.0,1.0,0.0,2.000000,2.0,2.0,0.0
2797,3.0,1.0,1.0,4.0,1.0,1.0,0.0,2.000000,2.0,2.0,0.0
2798,3.0,1.0,1.0,5.0,1.0,1.0,0.0,2.000000,2.0,2.0,0.0
2799,3.0,1.0,1.0,6.0,1.0,1.0,0.0,2.000000,2.0,2.0,0.0
2800,3.0,1.0,1.0,7.0,1.0,1.0,0.0,2.000000,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
28051,2.0,1.0,7.0,7.0,5.0,5.0,16.0,5.000000,7.0,4.0,3.0
28052,2.0,1.0,7.0,7.0,5.0,6.0,16.0,5.830952,8.0,5.0,2.0
28053,2.0,1.0,7.0,7.0,5.0,7.0,16.0,6.708204,9.0,6.0,1.0
28054,2.0,1.0,7.0,7.0,6.0,5.0,16.0,5.656854,8.0,4.0,2.0


### Wykresy

In [177]:


def wykresMetrykaOdległość(df, metryka):
    fig = go.Figure()
    czasyDoWygranej = sorted(df["white-depth-of-win"].unique())
    mapa_x = {d: i for i, d in enumerate(czasyDoWygranej)}

    odstępy = [-0.3, -0.1, 0.1, 0.3] 

    fig.add_trace(go.Box(
        y=df.loc[df["Do krawędzi"]==0, metryka],
        x=df.loc[df["Do krawędzi"]==0, "white-depth-of-win"].map(mapa_x) + odstępy[0],
        name='Odległość od krawędzi 0',
        marker_color='#3D9970',
        width=0.2
    ))
    
    fig.add_trace(go.Box(
        y=df.loc[df["Do krawędzi"]==1, metryka],
        x=df.loc[df["Do krawędzi"]==1, "white-depth-of-win"].map(mapa_x) + odstępy[1],
        name='Odległość od krawędzi 1',
        marker_color="#1502BD",
        width=0.2
    ))
    
    fig.add_trace(go.Box(
        y=df.loc[df["Do krawędzi"]==2, metryka],
        x=df.loc[df["Do krawędzi"]==2, "white-depth-of-win"].map(mapa_x) + odstępy[2],
        name='Odległość od krawędzi 2',
        marker_color="#FF00C8",
        width=0.2
    ))
    
    fig.add_trace(go.Box(
        y=df.loc[df["Do krawędzi"]==3, metryka],
        x=df.loc[df["Do krawędzi"]==3, "white-depth-of-win"].map(mapa_x) + odstępy[3],
        name='Odległość od krawędzi 3',
        marker_color="#D30808",
        width=0.2
    ))

    for ileDoWygranej in czasyDoWygranej:
        idx = mapa_x[ileDoWygranej]
        y_vals = []
        x_vals = []
        
        for krawedz in range(4):
            mask = (df["white-depth-of-win"] == ileDoWygranej) & (df["Do krawędzi"] == krawedz)
            if any(mask):
                y_vals.append(df.loc[mask, metryka].median())
                x_vals.append(idx + odstępy[krawedz])
        
        fig.add_trace(go.Scatter(
            x=x_vals, y=y_vals,
            mode='lines', line=dict(color='black', width=3),
            showlegend=False, hoverinfo='skip'
        ))

    fig.update_layout(
        yaxis=dict(title=dict(text=f'Odległość w {metryka}')),
        boxmode='overlay', 
        width=1600, height=500,
        xaxis=dict(
            tickmode='array',
            tickvals=list(mapa_x.values()),
            ticktext=list(mapa_x.keys()),
            title='White ileDoWygranej of Win'
        )
    )
    fig.show()

In [178]:
wykresMetrykaOdległość(df3,"Euklides")

In [179]:
wykresMetrykaOdległość(df3,"Manhattan")

In [180]:
wykresMetrykaOdległość(df3,"Czebyszew")

## Sama odległość od krawędzi nie jest zbyt przydatna w porównaniu do metryk. Gdy uzwględnimy rozkłady łączne odległości od krawędzi oraz każdej metryki z osobna, możemy zauważyć iż najbardziej modelowo w stosunku do wybranej metryki, wyglądą połączenie odległości Czebyszewa między królami z dystansem od krawędzi czarnego króla. Objawia się to w najbardziej równoległych liniach wytyczonych między medianami na boxplotach (Efektem temu sprzyjającym może być również niższa różnorodność możliwych wartości metryki Czebyszewa 2-7 vs 2-10 i 2-14). Dzięki temu powinniśmy być w stanie lepiej oszacować odległość od mata na podstawie metryki Czebyszewa przy użyciu informacji o odległości czarnego króla od krawędzi.

In [181]:
print(dopasowania[["Metryka", "R2"]].sort_values(by="R2", ascending=False))

       Metryka        R2
1    Manhattan  0.921601
0     Euklides  0.916292
2    Czebyszew  0.813225
3  Do krawędzi  0.611413
